In [232]:
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential, EnvironmentCredential
from azure.ai.ml.entities import AmlCompute
from dotenv import load_dotenv
import os
import pandas as pd

In [233]:
#Omitir este paso en azure
print("Cargar username/pass desde archivo .env")
load_dotenv("env.txt", override=True)

Cargar username/pass desde archivo .env


True

In [234]:
def get_comput_target(ml_client, name="cpu-cluster", family='Standard_DS2_v2'):
    cpu_compute_target = name
    
    try:
        # let's see if the compute target already exists
        cpu_cluster = ml_client.compute.get(cpu_compute_target)
    except Exception:
        cpu_cluster = AmlCompute(
            name=cpu_compute_target,
            type="amlcompute",
            size=family,
            min_instances=0,
            max_instances=4,
            idle_time_before_scale_down=180,
            tier="Dedicated",
        )
    
        cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

In [235]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: .\config.json


In [236]:
#Omitir este paso en azure
credential.get_token("https://management.azure.com/.default")

AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjlHbW55RlBraGMzaE91UjIybXZTdmduTG83WSIsImtpZCI6IjlHbW55RlBraGMzaE91UjIybXZTdmduTG83WSJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvY2MyODYzM2YtMTJiOC00NmNiLWJjMTUtOTUxZGFlMjM5YjRkLyIsImlhdCI6MTY5OTQ3OTMxNywibmJmIjoxNjk5NDc5MzE3LCJleHAiOjE2OTk0ODMyMTcsImFpbyI6IkUyVmdZUGl5UmMweGNibFgwdklHOFRrejhqaVpBQT09IiwiYXBwaWQiOiJhMjcwZGI3OC1lODMwLTQxZTYtOTY5My1kMjg2N2I2ZWI4NWYiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9jYzI4NjMzZi0xMmI4LTQ2Y2ItYmMxNS05NTFkYWUyMzliNGQvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiJkY2VmMmQ2OS03NDAzLTRlYWUtODJmYy0yM2U0MjhmZjA5MmUiLCJyaCI6IjAuQVFvQVAyTW96TGdTeTBhOEZaVWRyaU9iVFVaSWYza0F1dGRQdWtQYXdmajJNQk1LQUFBLiIsInN1YiI6ImRjZWYyZDY5LTc0MDMtNGVhZS04MmZjLTIzZTQyOGZmMDkyZSIsInRpZCI6ImNjMjg2MzNmLTEyYjgtNDZjYi1iYzE1LTk1MWRhZTIzOWI0ZCIsInV0aSI6InJ3akRxdEllcUVhbU54LUdZQVZPQVEiLCJ2ZXIiOiIxLjAiLCJ4bXNfY2FlIjoiMSIsInhtc190Y2R0IjoxNDY1ODQ5MzEyfQ.uBUGVXAMMqDwNNe5t

In [237]:
compute_target = get_comput_target(ml_client)

In [238]:
clean_component = load_component(source="./components/dataset_clean/dataset_clean.yaml")
split_data_component = load_component(source="./components/split_data/split_data.yaml")
exploration_component = load_component(source="./components/data_exploration/data_exploration.yaml")
train_logistic_regression_component = load_component(source="./components/train_logistic_regression/train_logistic_regression.yaml")
train_decission_tree_component = load_component(source="./components/train_decission_tree/train_decission_tree.yaml")
score_model_component = load_component(source="./components/score_model/score_model.yaml")
evaluate_model_component = load_component(source="./components/evaluate_model/evaluate_model.yaml")

In [239]:
@pipeline(
    default_compute='cpu-cluster',
)
def water_potability_logistic_regression(pipeline_input_data):

    clean_node = clean_component(
        clean_data=pipeline_input_data,
    )
    
    exploration_node = exploration_component(
        exploration_data=clean_node.outputs.cleaned_data
    )
    
    split_node = split_data_component(
        data_path=clean_node.outputs.cleaned_data,
        test_size=0.2,
        random_state=42,
    )
    
    train_lr_node = train_logistic_regression_component(
        train_data_path=split_node.outputs.train_data,
    )

    score_node = score_model_component(
        test_data_path=split_node.outputs.test_data,
        model_path=train_lr_node.outputs.trained_model,
    )
    
    evaluate_node = evaluate_model_component(
        predictions_csv=score_node.outputs.predictions_csv,
        true_labels_csv=split_node.outputs.test_labels,
    )

    return {
        "pipeline_logistic_regression_trained_model": train_lr_node.outputs.trained_model,
        "pipeline_logistic_regression_evaluation_report": evaluate_node.outputs.report_csv,
        "pipeline_logistic_regression_confusion_matrix": evaluate_node.outputs.confusion_matrix
    }

water_potability_ds =  Input(
            type="uri_file",
            path="azureml://subscriptions/ba1f7bf8-2be6-4bed-b818-c745bda74905/resourcegroups/primer_proyecto/workspaces/primer_proyecto/datastores/workspaceblobstore/paths/UI/2023-11-06_001630_UTC/water_potability_ds.csv",
        )
pipeline_job = water_potability_logistic_regression(pipeline_input_data=water_potability_ds)


In [240]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_water_potability"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_water_potability,coral_papaya_sm68nlqcdy,pipeline,Preparing,Link to Azure Machine Learning studio


In [241]:
# wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: coral_papaya_sm68nlqcdy
Web View: https://ml.azure.com/runs/coral_papaya_sm68nlqcdy?wsid=/subscriptions/ba1f7bf8-2be6-4bed-b818-c745bda74905/resourcegroups/primer_proyecto/workspaces/primer_proyecto

Execution Summary
RunId: coral_papaya_sm68nlqcdy
Web View: https://ml.azure.com/runs/coral_papaya_sm68nlqcdy?wsid=/subscriptions/ba1f7bf8-2be6-4bed-b818-c745bda74905/resourcegroups/primer_proyecto/workspaces/primer_proyecto



In [242]:
# Download all the outputs of the job
output = ml_client.jobs.download(name=pipeline_job.name, download_path='./pipeline_output', all=True)

In [217]:
@pipeline(
    default_compute='cpu-cluster',
)
def water_potability_decission_tree(pipeline_input_data):

    clean_node = clean_component(
        clean_data=pipeline_input_data,
    )
    
    exploration_node = exploration_component(
        exploration_data=clean_node.outputs.cleaned_data
    )
    
    split_node = split_data_component(
        data_path=clean_node.outputs.cleaned_data,
        test_size=0.2,
        random_state=42,
    )
    
    train_dt_node = train_decission_tree_component(
        train_data_path=split_node.outputs.train_data,
    )

    score_node = score_model_component(
        test_data_path=split_node.outputs.test_data,
        model_path=train_dt_node.outputs.trained_model,
    )
    
    evaluate_node = evaluate_model_component(
        predictions_csv=score_node.outputs.predictions_csv,
        true_labels_csv=split_node.outputs.test_labels,
    )

    return {
        "pipeline_decission_tree_trained_model": train_dt_node.outputs.trained_model,
        "pipeline_decission_tree_evaluation_report": evaluate_node.outputs.report_csv,
        "pipeline_decission_tree_confusion_matrix": evaluate_node.outputs.confusion_matrix
    }


# create a pipeline
water_potability_ds =  Input(
            type="uri_file",
            path="azureml://subscriptions/ba1f7bf8-2be6-4bed-b818-c745bda74905/resourcegroups/primer_proyecto/workspaces/primer_proyecto/datastores/workspaceblobstore/paths/UI/2023-11-06_001630_UTC/water_potability_ds.csv",
        )
pipeline_job = water_potability_decission_tree(pipeline_input_data=water_potability_ds)

In [218]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_water_potability"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_water_potability,gentle_iron_nq8c81gqbw,pipeline,Preparing,Link to Azure Machine Learning studio


In [219]:
# wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: gentle_iron_nq8c81gqbw
Web View: https://ml.azure.com/runs/gentle_iron_nq8c81gqbw?wsid=/subscriptions/ba1f7bf8-2be6-4bed-b818-c745bda74905/resourcegroups/primer_proyecto/workspaces/primer_proyecto

Execution Summary
RunId: gentle_iron_nq8c81gqbw
Web View: https://ml.azure.com/runs/gentle_iron_nq8c81gqbw?wsid=/subscriptions/ba1f7bf8-2be6-4bed-b818-c745bda74905/resourcegroups/primer_proyecto/workspaces/primer_proyecto



In [220]:
# Download all the outputs of the job
output = ml_client.jobs.download(name=pipeline_job.name, download_path='./pipeline_output', all=True)